In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
dataset=pd.read_csv("CKD.CSV")

In [3]:
dataset=pd.get_dummies(dataset,dtype=int,drop_first=True)

In [4]:
dataset.columns

Index(['age', 'bp', 'al', 'su', 'bgr', 'bu', 'sc', 'sod', 'pot', 'hrmo', 'pcv',
       'wc', 'rc', 'sg_b', 'sg_c', 'sg_d', 'sg_e', 'rbc_normal', 'pc_normal',
       'pcc_present', 'ba_present', 'htn_yes', 'dm_yes', 'cad_yes',
       'appet_yes', 'pe_yes', 'ane_yes', 'classification_yes'],
      dtype='object')

In [5]:
independent=dataset[['age', 'bp', 'al', 'su', 'bgr', 'bu', 'sc', 'sod', 'pot', 'hrmo', 'pcv',
       'wc', 'rc', 'sg_b', 'sg_c', 'sg_d', 'sg_e', 'rbc_normal', 'pc_normal',
       'pcc_present', 'ba_present', 'htn_yes', 'dm_yes', 'cad_yes',
       'appet_yes', 'pe_yes', 'ane_yes']]

In [6]:
dependent=dataset[['classification_yes']]

In [7]:
#from sklearn.model_selection import train_test_split
#x_train,x_test,y_train,y_test=train_test_split(independent,dependent,test_size=1/3,random_state=0)

In [8]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
independent= sc.fit_transform(independent)
#x_test_= sc.transform(x_test)

In [9]:
import pickle
xfilename="svc_model.pkl"
pickle.dump(sc,open("svc_model.pkl",'wb'))
sc=pickle.load(open("svc_model.pkl",'rb'))

In [10]:
from sklearn.svm import SVC

In [11]:
from sklearn.model_selection import GridSearchCV

param_grid = {'kernel':['linear','rbf','poly','sigmoid'],
             'gamma':['auto','scale'],
             'C':[10,50,100,200,300]} 



grid = GridSearchCV(SVC(probability=True), param_grid, refit = True, verbose = 3,n_jobs=-1,scoring='f1_weighted') 
   

grid.fit(independent,dependent)

Fitting 5 folds for each of 40 candidates, totalling 200 fits


C:\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GridSearchCV(estimator=SVC(probability=True), n_jobs=-1,
             param_grid={'C': [10, 50, 100, 200, 300],
                         'gamma': ['auto', 'scale'],
                         'kernel': ['linear', 'rbf', 'poly', 'sigmoid']},
             scoring='f1_weighted', verbose=3)

In [12]:
re=grid.cv_results_
grid_predictions = grid.predict(independent) 
   
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(dependent, grid_predictions)

from sklearn.metrics import classification_report
clf_report = classification_report(dependent, grid_predictions)




In [13]:
from sklearn.metrics import f1_score
f1_macro=f1_score(dependent,grid_predictions,average='weighted')
print("The f1_macro value for best parameter {}:".format(grid.best_params_),f1_macro)


The f1_macro value for best parameter {'C': 10, 'gamma': 'auto', 'kernel': 'poly'}: 1.0


In [14]:
print("The confusion Matrix:\n",cm)

The confusion Matrix:
 [[150   0]
 [  0 249]]


In [15]:
print("The report:\n",clf_report)

The report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00       150
           1       1.00      1.00      1.00       249

    accuracy                           1.00       399
   macro avg       1.00      1.00      1.00       399
weighted avg       1.00      1.00      1.00       399



In [16]:
from sklearn.metrics import roc_auc_score
roc_auc_score(dependent,grid.predict_proba(independent)[:,1])

1.0

In [17]:
table=pd.DataFrame.from_dict(re)

In [22]:
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_gamma,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.038775,0.018702,0.026184,0.002313,10,auto,linear,"{'C': 10, 'gamma': 'auto', 'kernel': 'linear'}",0.802005,0.975000,0.987539,1.000000,0.987298,0.950369,0.074602,31
1,0.026183,0.001719,0.028781,0.005265,10,auto,rbf,"{'C': 10, 'gamma': 'auto', 'kernel': 'rbf'}",0.926015,0.987539,1.000000,1.000000,0.987298,0.980171,0.027656,5
2,0.048169,0.001164,0.025585,0.002496,10,auto,poly,"{'C': 10, 'gamma': 'auto', 'kernel': 'poly'}",0.950521,0.987539,1.000000,1.000000,0.987298,0.985072,0.018169,1
3,0.022785,0.001599,0.023785,0.002134,10,auto,sigmoid,"{'C': 10, 'gamma': 'auto', 'kernel': 'sigmoid'}",0.913778,0.987539,1.000000,1.000000,0.987298,0.977723,0.032464,7
4,0.021785,0.002225,0.023586,0.001356,10,scale,linear,"{'C': 10, 'gamma': 'scale', 'kernel': 'linear'}",0.802005,0.975000,0.987539,1.000000,0.987298,0.950369,0.074602,31
5,0.024584,0.003497,0.025184,0.002039,10,scale,rbf,"{'C': 10, 'gamma': 'scale', 'kernel': 'rbf'}",0.926015,0.987539,1.000000,1.000000,0.987298,0.980171,0.027656,5
6,0.057164,0.010296,0.024385,0.002653,10,scale,poly,"{'C': 10, 'gamma': 'scale', 'kernel': 'poly'}",0.950521,0.987539,1.000000,1.000000,0.987298,0.985072,0.018169,1
7,0.023786,0.001599,0.023785,0.000748,10,scale,sigmoid,"{'C': 10, 'gamma': 'scale', 'kernel': 'sigmoid'}",0.913778,0.987539,1.000000,1.000000,0.987298,0.977723,0.032464,7
8,0.019987,0.002190,0.022786,0.000748,50,auto,linear,"{'C': 50, 'gamma': 'auto', 'kernel': 'linear'}",0.802005,0.975000,0.987539,1.000000,0.987298,0.950369,0.074602,31
9,0.023585,0.003439,0.025185,0.003369,50,auto,rbf,"{'C': 50, 'gamma': 'auto', 'kernel': 'rbf'}",0.913778,0.987539,1.000000,1.000000,0.987298,0.977723,0.032464,7


In [23]:
import pickle
filename="final_svc_model.sav"
pickle.dump(grid,open("final_svc_model.sav",'wb'))

In [19]:
preinput=sc.transform([[51,70,0,0,219,36,1.3,139,3.7,12.5,49,6600,4.705,0,0,1,1,0,1,1,0,1,0,0,1,1,0]])

C:\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [20]:
preinput

array([[-2.90035181e-02, -4.79397659e-01, -6.85199355e-01,
        -3.80814665e-01,  9.48067301e-01, -4.35970806e-01,
        -3.16440999e-01,  1.59843796e-01, -3.28835132e-01,
        -6.69369132e-03,  1.24353032e+00, -7.16672489e-01,
        -7.10773454e-04, -5.04694939e-01, -5.16397779e-01,
         2.07846097e+00,  7.48331477e+00, -2.73666982e+00,
         4.85071250e-01,  2.91547595e+00, -2.41568702e-01,
         1.31638775e+00, -7.19103794e-01, -3.05205971e-01,
         5.12501930e-01,  2.04494943e+00, -4.20703162e-01]])

In [21]:
Future_Prediction=grid.predict(preinput) 
print("Future_Prediction={}".format(Future_Prediction))

Future_Prediction=[1]


In [24]:
loaded_model=pickle.load(open("final_svc_model.sav",'rb'))
result=loaded_model.predict(preinput)


In [25]:
result

array([1])